In [ ]:
def run_lstm(data_shape, batched_train_data, batched_val_data, train_slices, 
             val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET):
    model = compile_model(data_shape, FUTURE_TARGET)
    training_history, training_time = fit_model(model, batched_train_data, batched_val_data, train_slices, 
                                                val_slices, BATCH_SIZE, EPOCHS)
    return model, training_history, training_time

In [ ]:
def compile_model(data_shape, FUTURE_TARGET):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(32, input_shape=data_shape, return_sequences = False))
    model.add(tf.keras.layers.Dense(FUTURE_TARGET))
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae',metrics=['mae', 'mape', 'mse'])
    model.summary()
    return model

In [ ]:
def fit_model(model, batched_train_data, batched_val_data, train_slices, val_slices, BATCH_SIZE, EPOCHS):
    train_batches = train_slices // BATCH_SIZE 
    val_batches = val_slices // BATCH_SIZE
    # Splits the dataset into batches of this size: we perform gradiant descent once per batch
    start = timer()
    training_history = model.fit(batched_train_data, epochs=EPOCHS, 
                                 steps_per_epoch=train_batches,
                                 validation_data=batched_val_data,
                                 validation_steps=val_batches)
    end = timer()
    return training_history, end - start # time in seconds